In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts

In [3]:
##extracting anticoag info as requested by user
sql = '''
SELECT
  month,
  presc.practice,
  prac.name, 
  prac.postcode,
  bnf_code AS BNFCode,
  bnf_name AS BNFName,
  SUM(items) AS TotalItems,
  SUM(actual_cost) AS TotalCost  
FROM
  ebmdatalab.hscic.normalised_prescribing_standard AS presc
JOIN
  ebmdatalab.hscic.practices prac
ON
  presc.practice = prac.code
WHERE
  (bnf_code LIKE "0208020Y0%" ##rivaroxaban1
  OR bnf_code LIKE "0208020X0%" ##dabigatran1
  OR bnf_code LIKE "0208020Z0%" ##apixaban1
  OR bnf_code LIKE "0208020AA%" ##edoxaban1
  OR bnf_code LIKE "0208020V0%") ##warfarin1
  AND
  (month >= "2018-04-01")
GROUP BY
  month,
presc.practice,
  prac.name,  
  prac.postcode,
  bnf_code,
  bnf_name
  '''

df_anticoag_jb = bq.cached_read(sql, csv_path='anticoagjb.csv')
df_anticoag_jb.head(50)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,month,practice,name,postcode,BNFCode,BNFName,TotalItems,TotalCost
0,2018-08-01,L83048,ROBOROUGH SURGERY,PL6 6PH,0208020X0AAAAAA,Dabigatran Etexilate_Cap 75mg,1,44.32137
1,2019-03-01,J82073,KIRKLANDS SURGERY,PO3 5AF,0208020Z0BBAAAA,Eliquis_Tab 2.5mg,2,99.07645
2,2019-01-01,F81183,THE DENGIE MEDICAL PARTNERSHIP,CM0 7TH,0208020Z0BBAAAA,Eliquis_Tab 2.5mg,2,105.80884
3,2018-09-01,A82036,THE LAKES MEDICAL PRACTICE,CA11 8HW,0208020Z0BBABAB,Eliquis_Tab 5mg,3,37.35714
4,2018-04-01,P81061,DR A CALOW'S PRACTICE,BB2 1UW,0208020X0BBACAC,Pradaxa_Cap 150mg,1,47.30937
5,2019-03-01,C82109,HUSBANDS BOSWORTH MEDICAL CENTRE,LE17 6PU,0208020Z0BBAAAA,Eliquis_Tab 2.5mg,1,52.95624
6,2019-02-01,B82050,MOWBRAY HOUSE SURGERY,DL7 8FW,0208020X0BBABAB,Pradaxa_Cap 110mg,9,426.23655
7,2018-04-01,C83062,CHURCH WALK SURGERY,LN4 3EZ,0208020X0AAAAAA,Dabigatran Etexilate_Cap 75mg,1,47.30937
8,2018-04-01,D82070,GREAT MASSINGHAM SURGERY,PE32 2JQ,0208020Z0BBAAAA,Eliquis_Tab 2.5mg,1,52.86132
9,2018-11-01,D81062,HADDENHAM SURGERY,CB6 3TA,0208020Z0BBAAAA,Eliquis_Tab 2.5mg,4,90.23533
